Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [ ]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [ ]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [ ]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [ ]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [ ]:
# adding any imports i need here

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score
import category_encoders as cat

In [ ]:
df['Burrito'].value_counts()

In [ ]:
df_copy = df.copy()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
# for sure dropping queso, could drop anything from taquito - zucchini, honestly, but we won't
df = df.drop(['Queso'], axis=1)   # that should be good enough for now

In [ ]:
# Make all the dtypes nice for logistic regression

df = df.replace({'x': 1, 'X': 1, 'yes': 1, 'Yes': 1, 'No': 0, 'no': 0, True: 1, False: 0, 'Other': 0})

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], infer_datetime_format=True)

print(df['Date'].min())
print(df['Date'].max())

In [ ]:
df['Date'] = df['Date'].dt.year   # COMMENT THIS OUT ONCE YOU RUN ONCE
train = df[df['Date'] == 2016]
test = df[(df['Date'] == 2018) | (df['Date'] == 2019)]
val = df[df['Date'] == 2017]

print('train shape: ', train.shape)
print('test shape: ', test.shape)
print('validate shape: ', val.shape)

In [ ]:
# Declare baseline and target

target = 'Great'
baseline = df[target].value_counts(normalize=True)

In [ ]:
burrito_list = ['Burrito_California', 'Burrito_Carnitas', 'Burrito_Asada', 'Burrito_0',
       'Burrito_Surf & Turf']

In [ ]:
# Features list

features = ['Burrito', 'Date', 'Yelp', 'Google', 'Chips', 'Cost', 'Hunger',
            'Mass (g)', 'Density (g/mL)', 'Length', 'Circum', 'Volume', 'Tortilla',
            'Temp', 'Meat', 'Fillings', 'Meat:filling', 'Uniformity', 'Salsa', 
            'Wrap', 'Unreliable', 'NonSD', 'Beef', 'Pico', 'Guac',
            'Cheese', 'Fries', 'Sour cream', 'Pork', 'Chicken', 'Shrimp', 'Fish',
            'Rice', 'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots',
            'Cabbage', 'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito',
            'Pineapple', 'Ham', 'Chile relleno', 'Nopales', 'Lobster', 'Egg',
            'Mushroom', 'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini']

# Regression variables

X_train = train[features]
X_test = test[features]
X_val = val[features]

y_train = train[target]
y_test = test[target]
y_val = val[target]

In [ ]:
# Instantiate OneHot, encode stuff

# encoder = cat.OneHotEncoder(use_cat_names=True)
# X_train = encoder.fit_transform(X_train)
# X_val = encoder.transform(X_val)
# X_test = encoder.transform(X_test)

# Logistic Regression

log_reg = LogisticRegression(solver='lbfgs')
log_reg.fit(X_train, y_train)
print('Validation Accuracy: ', log_reg.score(X_val, y_val))

# this is returning error: 'could not convert string to float: 'Asada''

# but below, Asada is an int?

In [ ]:
df['Burrito'].value_counts()

In [ ]:
print('Test Accuracy', log_reg.score(X_test, y_test))